# Extracte

In [ ]:
import pandas as pd
import requests
import numpy as np
from sqlalchemy import create_engine, text


In [3]:
def extract_csv(list_csv):


    api_url = "https://www.data.gouv.fr/api/2/datasets/5b598be088ee387c0c353714/resources/?page=1&page_size=50" # La requette est sur le /states/all et c'est la que les param sont important et préciser
    

    
    try:
        print("Making API request... (this may take a few seconds)")
        


        response = requests.get(api_url, timeout=10)
        


        if response.status_code == 200: #status_code = le statu html de la reponse et 200 c'est quand c'est bon
            print("Cela fonctionne")



            data = response.json() # Vas cherche les données de la request (voir ca comme les ROS)
        
            for part in data.get('data', []):

                title = part.get('title')
                

                if title in  list_csv:
                    
                    url_csv  = part.get('url')

                    requests_csv = requests.get(url_csv)

                    if requests_csv.status_code == 200:

                        with open("data/"+ title, "wb") as f:
                            f.write(requests_csv.content)


        print(f"Created all CSV")
    
    except requests.exceptions.RequestException as e:
        print(f"❌ Network error fetching data: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"❌ Error processing data: {e}")
        return pd.DataFrame()

In [4]:
def extract_data(list_chemin):

    # print("📄 Reading tourism data from CSV...")
    
    try:

        df = pd.DataFrame()
        for chemin in list_chemin:
            df = pd.concat([df, pd.read_csv("../data/"+ chemin)], ignore_index=True)

        print('Data retreived in a dataframe')
        return df
        
    except Exception as e:
        print(f"❌ Error reading data: {e}")
        return pd.DataFrame()



In [5]:
list_df = ["datatourisme-reg-ara.csv", "datatourisme-reg-bfc.csv", "datatourisme-reg-bre.csv",
        "datatourisme-reg-cor.csv", "datatourisme-reg-cvl.csv", "datatourisme-reg-gde.csv",
        "datatourisme-reg-hdf.csv", "datatourisme-reg-naq.csv", "datatourisme-reg-nor.csv",
        "datatourisme-reg-idf.csv",  "datatourisme-reg-occ.csv", "datatourisme-reg-pac.csv",
        "datatourisme-reg-pdl.csv"] # "datatourisme-reg-guf", "datatourisme-reg-myt", , "datatourisme-reg-reu", "datatourisme-reg-glp","datatourisme-reg-mtq",

In [11]:
extract_csv(list_df)

df = extract_data(list_df)

Making API request... (this may take a few seconds)
Cela fonctionne
❌ Error processing data: [Errno 2] No such file or directory: 'data/datatourisme-reg-pdl.csv'
❌ Error reading data: [Errno 2] No such file or directory: '../data/datatourisme-reg-ara.csv'


# Transforme

In [ ]:
### Cluster ###



# A garder score cacher qui compte sans un poids du client
Logement = ['Hotel', 'BedAndBreakfast', 'HotelRestaurant', 'Hostel', 'CampingAndCaravanning',
            'Accommodation', 'HotelTrade', 'RentalAccommodation', 'CollectiveAccommodation', 'TableHoteGuesthouse',
            "AccommodationProduct", 'Guesthouse', 'House'] # LodgingBusiness = logement qui accepte les buisness

# A garder score cacher qui compte sans un poids du client
tourism_center = ["TouristInformationCenter"]

# Garder
Nourriture = ["FoodEstablishment", 'Restaurant', 'CafeOrCoffeeShop', 'IceCreamShop', 'Bakery']

# Garder
Event = ['SaleEvent', 'TheaterEvent', 'Event', 'Festival', 'MusicEvent', "SportsEvent", 'TraditionalCelebration', 'ShowEvent', 'ChildrensEvent',
         'Concert', 'Exhibition', 'LocalAnimation', 'Rambling']

# A garder score cacher qui compte sans un poids du client
transport = ['Transport', 'TrainStation', 'BusStation', 'Transporter', 'Airport', 'TaxiCompany'] # Transport = principalement des ports/ BusStation = gare routière

# Garder
activités = ['Product', 'Hammam', 'AmusementPark', 'Landform', 'Casino',  'BowlingAlley', 'RailBike', 'MiniGolf', 'AdventurePark'
             'BalneotherapyCentre', 'SummerToboggan', 'NauticalCentre',
             'TastingProvider', 'ActivityProvider',  'Rental', 'Trampoline', 'EquestrianCenter', 'EquipmentRental',
             "Tour", 'LeisureSportActivityProvider', 'Practice', 'EntertainmentAndEvent', 'MegalithDolmenMenhir', 'TrainingWorkshop', 'TeachingFarm',
             "CulturalActivityProvider", 'Cinematheque', 'Visit', 'WalkingTour'] # Product = visite en tt genre / Landform = Plage /  Tour = sentier de rando

# A garder
Sport = ['SportsAndLeisurePlace', 'OrderedList',  'GolfCourse', 'ClimbingWall', 'TennisComplex', 
         "CyclingTour", 'TerrainPark', 'FrontonBelotaCourt', 'SportsClub'] # OrderedList = rando + VTT

# A garder
Sport_hiver = ['CrossCountrySkiTrail', 'DownhillSkiRun', 'DownhillSkiResort', 'CrossCountrySkiResort']


# A garder
Balade = ['NaturalHeritage', 'ServiceArea','EducationalTrail', 'ViaFerrata', 'RomanPath', 'LevyOrDike',] # ServiceArea = Site pour observer les étoile / EducationalTrail = balade / LevyOrDike = balade de barrage

# A garder
Park = ['Park', 'CivicStructure', 'PicnicArea', 'ParkAndGarden'] # CivicStructure = Parcoure de santé, air de jeux

Magasin = ['CoveredMarket', "Store", 'Market', 'LocalProductsShop', 'BoutiqueOrLocalShop']


# A garder
Culture = ['Church', 'RemarkableBuilding', 'TechnicalHeritage', 'Cloister', 'Cathedral', 'FortifiedCastle', 'Palace', 
            'Fort', 'ReligiousSite', 'Temple', "Ruins", "RemembranceSite", 'Dungeon', 'DefenceSite', 'Abbey', 'Convent', 
            'Monastery', 'Collegiate', 'Tower', 'Fountain','Chapel', 'Mine', 'Bridge', 'Basilica', 'Chartreuse',
            'BuddhistTemple', 'Mosque', 'Aqueduct', 'ArcheologicalSite',"Castle", 'Synagogue', 'FortifiedSet', 'Citadel', 
            "RemarkableHouse", "Commanderie", 'Marina', 'Bastide', 'Lighthouse','Arena', 'LocalBusiness', 'Aquarium', 
            "CulturalSite", 'Theater', "Library", 'Museum'] # RemembranceSite = Memoriale,...

# A garder
Sortie_soir = ['Winery', 'NightClub', 'BistroOrWineBar', 'BrasserieOrTavern']

In [ ]:
Liste_to_keep = ['Winery', 'NightClub', 'BistroOrWineBar', 'BrasserieOrTavern', 'Church', 'RemarkableBuilding', 'TechnicalHeritage', 
           'Cloister', 'Cathedral', 'FortifiedCastle', 'Palace', 'BalneotherapyCentre', 'SummerToboggan', 'NauticalCentre',
            'Fort', 'ReligiousSite', 'Temple', "Ruins", "RemembranceSite", 'Dungeon', 'DefenceSite', 'Abbey', 'Convent', 
            'Monastery', 'Collegiate', 'Tower', 'Fountain','Chapel', 'Mine', 'Bridge', 'Basilica', 'Chartreuse',
            'BuddhistTemple', 'Mosque', 'Aqueduct', 'ArcheologicalSite',"Castle", 'Synagogue', 'FortifiedSet', 'Citadel', 
            "RemarkableHouse", "Commanderie", 'Marina', 'Bastide', 'Lighthouse','Arena', 'LocalBusiness', 'Aquarium', 
            "CulturalSite", 'Theater', "Library", 'Museum', 'Hotel', 'BedAndBreakfast', 'HotelRestaurant', 'Hostel', 'CampingAndCaravanning',
            'Accommodation', 'HotelTrade', 'RentalAccommodation', 'CollectiveAccommodation', 'TableHoteGuesthouse',
            "AccommodationProduct", 'Guesthouse', 'House', "TouristInformationCenter", "FoodEstablishment", 'Restaurant', 'CafeOrCoffeeShop', 'IceCreamShop', 'Bakery',
            'SaleEvent', 'TheaterEvent', 'Event', 'Festival', 'MusicEvent', "SportsEvent", 'TraditionalCelebration', 'ShowEvent', 'ChildrensEvent',
            'Concert', 'Exhibition', 'LocalAnimation', 'Rambling', 'Transport', 'TrainStation', 'BusStation', 'Transporter', 'Airport', 'TaxiCompany',
            'Product', 'Hammam', 'AmusementPark', 'Landform', 'Casino',  'BowlingAlley', 'RailBike', 'MiniGolf', 'AdventurePark'
            'TastingProvider', 'ActivityProvider',  'Rental', 'Trampoline', 'EquestrianCenter', 'EquipmentRental',
            "Tour", 'LeisureSportActivityProvider', 'Practice', 'EntertainmentAndEvent', 'MegalithDolmenMenhir', 'TrainingWorkshop', 'TeachingFarm',
            "CulturalActivityProvider", 'Cinematheque', 'Visit', 'WalkingTour', 'SportsAndLeisurePlace', 'OrderedList',  'GolfCourse', 'ClimbingWall', 'TennisComplex', 
            "CyclingTour", 'TerrainPark', 'FrontonBelotaCourt', 'SportsClub', 'CrossCountrySkiTrail', 'DownhillSkiRun', 'DownhillSkiResort', 'CrossCountrySkiResort',
            'NaturalHeritage', 'ServiceArea','EducationalTrail', 'ViaFerrata', 'RomanPath', 'LevyOrDike','Park', 'CivicStructure', 'PicnicArea', 'ParkAndGarden',
            'CoveredMarket', "Store", 'Market', 'LocalProductsShop', 'BoutiqueOrLocalShop']

In [ ]:
categorie_dict = {
    "Logement": Logement,
    "Tourism_center": tourism_center,
    "Nourriture": Nourriture,
    "Event": Event,
    "Transport": transport,
    "Activités": activités,
    "Sport": Sport,
    "Sport_hiver": Sport_hiver,
    "Balade": Balade,
    "Park": Park,
    "Magasin": Magasin,
    "Culture": Culture,
    "Sortie_soir": Sortie_soir,
}

In [ ]:
def distance(lat1, lon1, lat2, lon2):
    R = 6371  
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

In [ ]:
def assign_nearest_cluster(df, cluster_points):

    poi_coords = df[["Latitude", "Longitude"]].to_numpy()
    clusters = np.array(cluster_points)



    distances = np.array([distance(poi_coords[:, 0], poi_coords[:, 1], lat_c, lon_c) for (lat_c, lon_c) in clusters]).T  

    df["id_cluster"] = np.argmin(distances, axis=1)

    return df

In [ ]:
def assigne_cluster(df, df_cluster):
    poi_code_postale = np.array(df['Code_postale'].values)

    cluster_code_postale = df_cluster[['code_postal', "zone_emploi"]].to_numpy()

    list_cluster = []
    for poi_code in poi_code_postale:
        for cluster_code in cluster_code_postale:
            if poi_code == int(cluster_code[0]):
                list_cluster.append(cluster_code[1])

    dict_to_add = {"Cluster_id" : list_cluster}
    df_temp = pd.DataFrame(dict_to_add)

    df.insert(2, "Cluster_id", df_temp["Cluster_id"])

    return df




In [ ]:
def clean_data(df_tourism, A_garder, categorie_dict, df_cluster):

    if df_tourism.empty:
        print("⚠️  No airport data to clean")
        return df_tourism
    
    print(f"🧹 Cleaning  data...")
    print(f"Starting with {len(df_tourism)} ")
    
    # Make a copy to avoid modifying the original
    df = df_tourism.copy()

    # Drop the usless data
    df.drop(['Periodes_regroupees', 'Covid19_mesures_specifiques', 'Contacts_du_POI', 'Classements_du_POI', 'SIT_diffuseur'], axis=1)


    # On récupère uniquement les catégorie de POI
    df["Categories_de_POI"] = df["Categories_de_POI"].str.split('/').str[-1]
    df["Categories_de_POI"] = df["Categories_de_POI"].str.split('#').str[-1]

    # On découpe la colone en 3 pour plus de clareter
    col_index = df.columns.get_loc("Code_postal_et_commune")

    df_temp = df["Code_postal_et_commune"].str.split('#', expand=True)
    df_temp.columns = ["Code_postale", "Commune"]
    df_temp["Département"] = df_temp["Code_postale"].str[0:2].astype(int)

    df.drop(columns=["Code_postal_et_commune"], inplace=True)


    df.insert(col_index, "Département", df_temp["Département"])
    df.insert(col_index + 1, "Code_postale", df_temp["Code_postale"])
    df.insert(col_index + 2, "Commune", df_temp["Commune"])

    # On drop les valeur null
    df = df.dropna(subset=["Nom_du_POI", "Categories_de_POI", 'Latitude', 'Longitude']) # ,"Adresse_postale"]) 

    # Passage des dates en format date
    df["Date_de_mise_a_jour"] = pd.to_datetime(df["Date_de_mise_a_jour"])

    

    # --- Étape 1 : filtrer selon A_garder ---
    df = df[df["Categories_de_POI"].isin(A_garder)].copy()

    # --- Étape 2 : ajouter la catégorie simplifiée ---
    def find_category(cat):
        for key, values in categorie_dict.items():
            if cat in values:
                return key
        return "Autre"  # si aucune correspondance trouvée

    df.insert(3, 'Categorie_simplifiee', df["Categories_de_POI"].apply(find_category)) 


    df["nb_nan"] = df.isna().sum(axis=1)

    # Trier pour que la ligne avec le moins de NaN soit en premier dans chaque groupe
    df = df.sort_values(by=["Nom_du_POI", "nb_nan"], ascending=[True, True])

    # Supprimer les doublons, en gardant la première (celle avec le moins de NaN)
    df = df.drop_duplicates(subset="Nom_du_POI", keep="first")

    # Supprimer la colonne temporaire
    df = df.drop(columns="nb_nan")

    # (optionnel) Réindexer proprement
    df = df.reset_index(drop=True)

    df.insert(0, 'ID', df["URI_ID_du_POI"].str.split('/').str[-1])

    df = df.sort_values(by=['Département'], ascending=[True]).reset_index(drop=True)
    
    df['Code_postale'] = df['Code_postale'].astype(float)

    df_cluster = df_cluster.rename(columns={'code_postal': 'Code_postale'})

    # Fusion (jointure) sur le code postal
    df_merged = df.merge(
        df_cluster[['Code_postale', 'zone_emploi']], 
        on='Code_postale', 
        how='left'  # garde tous les POI même sans cluster
    )

    # Renommer la colonne résultante pour plus de clarté
    df_merged = df_merged.rename(columns={'zone_emploi': 'Cluster_id'})

    print(f'Il reste {len(df)} data après nettoyage.') 

    return df

# Load

In [8]:
DATABASE_CONFIG = {
    'username': 'alexm',
    'password': '', 
    'host': 'localhost',
    'port': '5432',
    'database': 'airlife_db'
}

In [9]:
def get_connection_string():
    """Build PostgreSQL connection string"""
    return f"postgresql://{DATABASE_CONFIG['username']}:{DATABASE_CONFIG['password']}@{DATABASE_CONFIG['host']}:{DATABASE_CONFIG['port']}/{DATABASE_CONFIG['database']}"

In [10]:
def load_to_database(Tourisme_df):

    print("💾 Loading data to PostgreSQL database...")

    connection_string = get_connection_string()

    try:

        engine = create_engine(connection_string)

        Tourisme_df.to_sql("airports", con = engine, if_exists='replace', index=False)


        print(f"✅ Loaded {len(Tourisme_df)} airports to database")

    except Exception as e:
        print(f"❌ Error loading data to database: {e}")
        print("💡 Make sure:")
        print("   - PostgreSQL is running")
        print("   - Database 'airlife_db' exists") 
        print("   - Username and password are correct")
        print("   - Tables are created (run database_setup.sql)")